In [1]:
# ftle_frames_only.py
import numpy as np
import matplotlib.pyplot as plt
import os

# === CONFIGURATION ===
mode = "ftle_only"
T = 10           # FTLE integration window
h = 0.01
frame_dt = 0.1
periods = 30
omega = (6 * np.pi) / 10
A = 0.1
epsilon = 0.25

# === FUNCTIONS ===
def a(t): return epsilon * np.sin(omega * t)
def b(t): return 1 - 2 * epsilon * np.sin(omega * t)
def f(x, t): return a(t) * x**2 + b(t) * x
def df_dx(x, t): return 2 * a(t) * x + b(t)

def compute_flow(X, Y, t):
    fx = f(X, t)
    dfx = df_dx(X, t)
    x_dot = -np.pi * A * np.sin(np.pi * fx) * np.cos(np.pi * Y)
    y_dot = np.pi * A * np.cos(np.pi * fx) * np.sin(np.pi * Y) * dfx
    return x_dot, y_dot

def rk4_flow(X, Y, t, h):
    k1x, k1y = compute_flow(X, Y, t)
    k2x, k2y = compute_flow(X + 0.5*h*k1x, Y + 0.5*h*k1y, t + 0.5*h)
    k3x, k3y = compute_flow(X + 0.5*h*k2x, Y + 0.5*h*k2y, t + 0.5*h)
    k4x, k4y = compute_flow(X + h*k3x, Y + h*k3y, t + h)
    X_new = X + (h/6)*(k1x + 2*k2x + 2*k3x + k4x)
    Y_new = Y + (h/6)*(k1y + 2*k2y + 2*k3y + k4y)
    return X_new, Y_new

# === GRID SETUP ===
x_vals = np.linspace(0, 2, 300)
y_vals = np.linspace(0, 1, 150)
X0, Y0 = np.meshgrid(x_vals, y_vals)

# === OUTPUT SETUP ===
output_folder = f"ftle_frames"
os.makedirs(output_folder, exist_ok=True)

# === MAIN LOOP ===
t0_values = np.arange(0, periods * (10/3), frame_dt)

for frame_idx, t0 in enumerate(t0_values):
    t_max = t0 + T
    X, Y = X0.copy(), Y0.copy()
    
    # Advect grid for FTLE integration
    for t in np.arange(t0, t_max + h, h):
        X, Y = rk4_flow(X, Y, t, h)

    # Compute FTLE
    dx = x_vals[1] - x_vals[0]
    dy = y_vals[1] - y_vals[0]
    J11 = (X[2:,1:-1] - X[:-2,1:-1]) / (2*dx)
    J12 = (X[1:-1,2:] - X[1:-1,:-2]) / (2*dy)
    J21 = (Y[2:,1:-1] - Y[:-2,1:-1]) / (2*dx)
    J22 = (Y[1:-1,2:] - Y[1:-1,:-2]) / (2*dy)

    ftle = np.full(J11.shape, np.nan)
    for i in range(ftle.shape[0]):
        for j in range(ftle.shape[1]):
            J = np.array([[J11[i,j], J12[i,j]],
                          [J21[i,j], J22[i,j]]])
            delta = J.T @ J
            eigvals = np.linalg.eigvalsh(delta)
            max_eig = np.max(eigvals)
            if max_eig > 1e-12 and np.isfinite(max_eig):
                ftle[i,j] = (1.0/T) * np.log(np.sqrt(max_eig))

    ftle_full = np.full_like(X0, np.nan)
    ftle_full[1:-1, 1:-1] = ftle

    # Plot and save
    plt.figure(figsize=(12,6))
    plt.contourf(X0, Y0, np.ma.masked_invalid(ftle_full), levels=100, cmap='jet')
    plt.colorbar(label='FTLE')
    plt.title(f"FTLE Only | t={t0:.1f} → {t0+T:.1f}")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.xlim(0,2)
    plt.ylim(0,1)
    plt.gca().set_aspect('equal')
    filename = os.path.join(output_folder, f"ftle_{frame_idx:04d}.png")
    plt.savefig(filename, dpi=150)
    plt.close()
    print(f"Saved: {filename}")


Saved: ftle_frames/ftle_0000.png
Saved: ftle_frames/ftle_0001.png
Saved: ftle_frames/ftle_0002.png
Saved: ftle_frames/ftle_0003.png
Saved: ftle_frames/ftle_0004.png
Saved: ftle_frames/ftle_0005.png
Saved: ftle_frames/ftle_0006.png
Saved: ftle_frames/ftle_0007.png
Saved: ftle_frames/ftle_0008.png
Saved: ftle_frames/ftle_0009.png
Saved: ftle_frames/ftle_0010.png
Saved: ftle_frames/ftle_0011.png
Saved: ftle_frames/ftle_0012.png
Saved: ftle_frames/ftle_0013.png
Saved: ftle_frames/ftle_0014.png
Saved: ftle_frames/ftle_0015.png
Saved: ftle_frames/ftle_0016.png
Saved: ftle_frames/ftle_0017.png
Saved: ftle_frames/ftle_0018.png
Saved: ftle_frames/ftle_0019.png
Saved: ftle_frames/ftle_0020.png
Saved: ftle_frames/ftle_0021.png
Saved: ftle_frames/ftle_0022.png
Saved: ftle_frames/ftle_0023.png
Saved: ftle_frames/ftle_0024.png
Saved: ftle_frames/ftle_0025.png
Saved: ftle_frames/ftle_0026.png
Saved: ftle_frames/ftle_0027.png
Saved: ftle_frames/ftle_0028.png
Saved: ftle_frames/ftle_0029.png
Saved: ftl

In [2]:
# ftle_movie.py
import cv2
import os
import glob

frames_folder = "ftle_frames"
output_video = "ftle_movie.mp4"
fps = 20

# Get sorted list of frames
frames = sorted(glob.glob(os.path.join(frames_folder, "*.png")))

# Read first frame to get dimensions
frame_example = cv2.imread(frames[0])
height, width, layers = frame_example.shape

# Create video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

for frame_file in frames:
    img = cv2.imread(frame_file)
    video.write(img)

video.release()
print(f"Movie saved as {output_video}")


Movie saved as ftle_movie.mp4
